# Introduction to API Name Qualifying using Scalpel
The qualified API name is a string that represents the path from the top-level module to the object itself. The name format is defined in PEP 3155.

Python APIs or function names can be invoked in different ways depending on how they are imported. However, this results in inconveniences to API usage analysis. In this module, we will convert all function call names to their fully-qualified names which are dotted strings that can represent the path from the top-level module down to the object itself. Various tasks can be benefited from this functionality such as understanding deprecated API  usage, dependency parsing as well as building sound call graphs.

To perform the API name qualification, we use Mnode. A MNode class  is a representation of a Module as a node in a control flow graph (CFG).
It extracts information about the node, such as its id, its statements, function definitions, imports and its exits from the source code.

## Let's install Scalpel first

Use the command in your virtual environment to install Scalpel.
```console
pip install python-scalpel
```

Now, Lets import all the  necessary modules.

In [1]:
import os

from scalpel.core.mnode import MNode

Below is an example source code for which we will  generate fully qualified names of APIs initialized  in it.

In [2]:
source = """
import numpy as np
import pandas as pd
from random import choices

pd.read_csv("test.csv")
np.array([1,2,3,4,5,6])
data = [41, 50, 29]
means = sorted(mean(choices(data, k=len(data))) for i in range(100))

"""

We first create a MNode object with the name "local". The MNode object is used to represent the abstract syntax tree (AST) of the source code. The source code is first parsed using MNode and the AST is generated.

The next step is to parse the AST to extract the function calls and the imported name information.

In [3]:
mnode = MNode("local")
mnode.source = source
mnode.gen_ast()
# parse all function calls
func_calls = mnode.parse_func_calls()
# obtain the imported name information
import_dict = mnode.parse_import_stmts()

As the next step, we loop  through a list of function calls. For each function call, we extract all the function name and splits it into a list of parts. if the function calls are from a imported module, we join them using their full names in import dictionary. 

In [4]:
for call_info in func_calls:
    call_name = call_info["name"]
    dotted_parts = call_name.split(".")
    # if this function calls is from a imported module
    if dotted_parts[0] in import_dict:
        dotted_parts = [import_dict[dotted_parts[0]]] + dotted_parts[1:]
        call_name = ".".join(dotted_parts)
    print(call_name)

pandas.read_csv
numpy.array
sorted
mean
len
range
